In [1]:
!pip install "huggingface_hub[cli]" datasets transformers trl torch wandb ipywidgets ipykernel iprogress

In [2]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: embereagle to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
from datasets import load_dataset

# https://huggingface.co/datasets/HuggingFaceTB/smoltalk
dataset = load_dataset("HuggingFaceTB/smoltalk", 'all')

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['messages', 'source'],
        num_rows: 1043917
    })
    test: Dataset({
        features: ['messages', 'source'],
        num_rows: 54948
    })
})

In [5]:
dataset['train'][0]

{'messages': [{'content': 'The function \\( g(x) \\) satisfies the functional equation\n\\[ g(x + y) = g(x) + g(y) \\]\nfor all real numbers \\( x \\) and \\( y \\), and it is given that \\( g(3) = 4 \\). Find \\( g(10) \\).',
   'role': 'user'},
  {'content': 'Given the functional equation and the specific value \\( g(3) = 4 \\), we can find \\( g(1) \\) by using the equation multiple times:\n\\[\ng(3) = g(2) + g(1)\n\\]\n\\[\ng(2) = g(1) + g(1) = 2g(1)\n\\]\nThus,\n\\[\n4 = 2g(1) + g(1) = 3g(1)\n\\]\n\\[\ng(1) = \\frac{4}{3}\n\\]\nNow we can find \\( g(10) \\) using \\( g(1) \\):\n\\[\ng(10) = 10g(1) = 10 \\times \\frac{4}{3} = \\frac{40}{3}\n\\]\nHence, the value of \\( g(10) \\) is \\(\\boxed{\\frac{40}{3}}\\).',
   'role': 'assistant'}],
 'source': 'numina-cot-100k'}

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# https://huggingface.co/microsoft/DialoGPT-medium
# model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
# tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# https://huggingface.co/Qwen/Qwen2.5-0.5B
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B")
# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B", use_fast=True)

# https://huggingface.co/distilbert/distilgpt2
model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2", use_fast=True)

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
import multiprocessing
num_proc = multiprocessing.cpu_count()

def chatml_tokenize(batch):
    texts = []
    for messages in batch["messages"]:
        chat = ""
        for msg in messages:
            if msg["role"] == "user":
                chat += "<|user|> " + msg["content"].strip() + " " + tokenizer.eos_token + " "
            elif msg["role"] == "assistant":
                chat += "<|assistant|> " + msg["content"].strip() + " " + tokenizer.eos_token + " "
        texts.append(chat.strip())
    return tokenizer(texts, padding=False, truncation=False)

tokenized_train = dataset["train"].map(
    chatml_tokenize, batched=True, batch_size=1000, num_proc=num_proc, remove_columns=["messages"]
)
tokenized_test = dataset["test"].map(
    chatml_tokenize, batched=True, batch_size=1000, num_proc=num_proc, remove_columns=["messages"]
)

Map (num_proc=16):   0%|          | 0/1043917 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1271 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1600 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1228 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1060 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1869 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence leng

Map (num_proc=16):   0%|          | 0/54948 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1481 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1484 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1049 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1634 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2265 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence leng

In [8]:
# Should show ChatML-formatted text
print("Sample training example:")
print(tokenizer.decode(tokenized_train[0]["input_ids"])) 
# Expected: "<|user|> ... <|assistant|> ..."

Sample training example:
<|user|> The function \( g(x) \) satisfies the functional equation
\[ g(x + y) = g(x) + g(y) \]
for all real numbers \( x \) and \( y \), and it is given that \( g(3) = 4 \). Find \( g(10) \). <|endoftext|> <|assistant|> Given the functional equation and the specific value \( g(3) = 4 \), we can find \( g(1) \) by using the equation multiple times:
\[
g(3) = g(2) + g(1)
\]
\[
g(2) = g(1) + g(1) = 2g(1)
\]
Thus,
\[
4 = 2g(1) + g(1) = 3g(1)
\]
\[
g(1) = \frac{4}{3}
\]
Now we can find \( g(10) \) using \( g(1) \):
\[
g(10) = 10g(1) = 10 \times \frac{4}{3} = \frac{40}{3}
\]
Hence, the value of \( g(10) \) is \(\boxed{\frac{40}{3}}\). <|endoftext|>


In [9]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print(device)

cuda


In [10]:
# Add special tokens to tokenizer
special_tokens = ["<|user|>", "<|assistant|>"]
tokenizer.add_special_tokens({"additional_special_tokens": special_tokens})
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(50259, 768)

In [11]:
# Evaluate WITHOUT ChatML formatting
def base_model_eval(question):
    encoded = tokenizer(question, return_tensors="pt").to(device)
    generated = model.generate(**encoded, max_new_tokens=20)
    return tokenizer.decode(generated[0], skip_special_tokens=True)

print("BEFORE TRAINING (Raw model):")
print(base_model_eval("The capital of France is"))
print(base_model_eval("What is the capital of France?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


BEFORE TRAINING (Raw model):


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The capital of France is the capital of the French Republic. The capital of France is the capital of the French Republic. The
What is the capital of France?






















In [12]:
import random

# sample random indices from the test set
random_indices = random.sample(range(len(tokenized_test)), 400)

# create a new Dataset with only those
sampled_eval_dataset = tokenized_test.select(random_indices)

In [13]:
from transformers import DataCollatorForLanguageModeling
from trl import SFTConfig, SFTTrainer

# Enable gradient checkpointing to reduce memory
# model.gradient_checkpointing_enable()
# model.config.use_cache = False

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding if not set

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# training on lightning studio cloud L4 GPU
training_args = SFTConfig(
    output_dir="./trainer_output",
    per_device_train_batch_size=16,
    max_steps=1000,
    learning_rate=1e-5,
    bf16=True,
    logging_steps=10,
    save_total_limit=2,  # Keep last 2 checkpoints
    save_strategy="steps",
    save_steps=100,
    eval_strategy="steps",
    eval_steps=50,
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    optim="adamw_torch_fused",
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    run_name="lightning_l4"
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=sampled_eval_dataset,
    data_collator=data_collator,
)

trainer.train()


Converting train dataset to ChatML:   0%|          | 0/1043917 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1043917 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1043917 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/400 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
50,2.907400,2.748845
100,2.763200,2.553327
150,2.592600,2.472359
200,2.601300,2.427890
250,2.539800,2.400502
300,2.547200,2.385157
350,2.537500,2.370286
400,2.396600,2.358006
450,2.578500,2.348926
500,2.372600,2.341397


TrainOutput(global_step=1000, training_loss=2.5505421867370606, metrics={'train_runtime': 1005.8606, 'train_samples_per_second': 15.907, 'train_steps_per_second': 0.994, 'total_flos': 4180748009472000.0, 'train_loss': 2.5505421867370606})

In [41]:
# Evaluate WITH ChatML formatting
def chatml_eval(question):
    formatted_prompt = f"<|user|> {question} <|assistant|>"
    encoded = tokenizer(formatted_prompt, return_tensors="pt").to(device)
    generated = model.generate(**encoded, max_new_tokens=100)
    return tokenizer.decode(generated[0], skip_special_tokens=True)

print("\nAFTER TRAINING (ChatML-formatted):")
print(chatml_eval("The capital of France is"))
print('...')
print(chatml_eval("What is the capital of France?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



AFTER TRAINING (ChatML-formatted):


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 The capital of France is , a city of                                                                                                
...
 What is the capital of France?  is a French language spoken by about 1,000 people. 

France is a country with a population of about 1,000 people. 

The capital of France is a city of about 1,000 people. 

The capital of France is a city of about 1,000 people. 

The capital of France is a city of about 1,000 people. 

The capital of France is a city of about 1,000 people. 




In [35]:
assert model.get_output_embeddings().weight.data_ptr() == model.get_input_embeddings().weight.data_ptr()

In [36]:
# total_steps_in_train_dataset = len(tokenized_train)

# # Continue training, use last checkpoint
# trainer.args.max_steps += total_steps_in_train_dataset - trainer.state.global_step
# trainer.train(resume_from_checkpoint=True)

In [37]:
# # Evaluate WITH ChatML formatting
# def chatml_eval(question):
#     formatted_prompt = f"<|user|> {question} <|assistant|>"
#     encoded = tokenizer(formatted_prompt, return_tensors="pt").to(device)
#     generated = model.generate(**encoded, max_new_tokens=100)
#     return tokenizer.decode(generated[0], skip_special_tokens=True)

# print("\nAFTER TRAINING (ChatML-formatted):")
# print(chatml_eval("The capital of France is"))
# print('...')
# print(chatml_eval("What is the capital of France?"))